In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd
import numpy as np
from pydataset import data

In [5]:
spark= SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/12 16:31:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/12 16:31:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# Read the case, department, and source data into their own spark dataframes.

In [6]:
# read case csv
case_df= spark.read.csv('case.csv', sep=',',header=True, inferSchema=True)



In [10]:
case_df.show(3, vertical= True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11          
 SLA_due_date         | 1/5/18 8:30          
 case_late            | NO                   
 num_days_late        | -2.0126041

In [8]:
#read source csv
source_df= spark.read.csv('source.csv', sep=',',header=True, inferSchema=True)

In [13]:
source_df.show(5,vertical= True)


-RECORD 0---------------------------
 source_id       | 100137           
 source_username | Merlene Blodgett 
-RECORD 1---------------------------
 source_id       | 103582           
 source_username | Carmen Cura      
-RECORD 2---------------------------
 source_id       | 106463           
 source_username | Richard Sanchez  
-RECORD 3---------------------------
 source_id       | 119403           
 source_username | Betty De Hoyos   
-RECORD 4---------------------------
 source_id       | 119555           
 source_username | Socorro Quiara   
only showing top 5 rows



In [9]:
# read department csv from local diretory
dept_df= spark.read.csv('dept.csv', sep=',',header=True, inferSchema=True)

In [14]:
dept_df.show(5, vertical= True)

-RECORD 0--------------------------------------
 dept_division          | 311 Call Center      
 dept_name              | Customer Service     
 standardized_dept_name | Customer Service     
 dept_subject_to_SLA    | YES                  
-RECORD 1--------------------------------------
 dept_division          | Brush                
 dept_name              | Solid Waste Manag... 
 standardized_dept_name | Solid Waste          
 dept_subject_to_SLA    | YES                  
-RECORD 2--------------------------------------
 dept_division          | Clean and Green      
 dept_name              | Parks and Recreation 
 standardized_dept_name | Parks & Recreation   
 dept_subject_to_SLA    | YES                  
-RECORD 3--------------------------------------
 dept_division          | Clean and Green N... 
 dept_name              | Parks and Recreation 
 standardized_dept_name | Parks & Recreation   
 dept_subject_to_SLA    | YES                  
-RECORD 4-------------------------------

# Let's see how writing to the local disk works in spark:

### - Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
### - Inspect your folder structure. What do you notice?

In [16]:
#source data saved to json format.
source_df.write.json('source_json', mode='overwrite')

In [18]:
# source data save to _csv
source_df.write.csv('source_csv', mode='overwrite')

- upon inspecting the files saved it actually created a folder inside of which there were multiple files.

## Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.



In [20]:
files=[case_df, source_df,dept_df]

In [23]:
# check the datatypes
for file in files:
    print(file)
    print('-------------------')
    print(file.printSchema())

DataFrame[case_id: int, case_opened_date: string, case_closed_date: string, SLA_due_date: string, case_late: string, num_days_late: double, case_closed: string, dept_division: string, service_request_type: string, SLA_days: double, case_status: string, source_id: string, request_address: string, council_district: int]
-------------------
root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)

None
DataFrame[source_id: st

#### Upon checking the types ,case_df some changes needs to be done. Other two(source and dept )df have appropriate types.

In [22]:
case_df.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row

22/09/12 17:19:02 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 206716 ms exceeds timeout 120000 ms
22/09/12 17:19:02 WARN SparkContext: Killing executors is not supported by current scheduler.


#### Things we can change in case_df spark dataframe

- change the data type for case id from a integer to string
- change the data type of the date to datatime stamp
- change the column name for SLA_due date to case due date
- convert case late and case closed to boolean data type

In [25]:
#rename the column SLA to case due date
case_df= case_df.withColumnRenamed('SLA_due_date','case_due_date')

In [27]:
# convert the date to datestamp
fmt = "M/d/yy H:mm"
case_df = (
    case_df.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_opened_date", fmt))
)

In [29]:
# check to see datestamp worked
case_df.select("case_opened_date", "case_closed_date", "case_due_date").show(2)

+-------------------+-------------------+-------------------+
|   case_opened_date|   case_closed_date|      case_due_date|
+-------------------+-------------------+-------------------+
|2018-01-01 00:42:00|2018-01-01 00:42:00|2018-01-01 00:42:00|
|2018-01-01 00:46:00|2018-01-01 00:46:00|2018-01-01 00:46:00|
+-------------------+-------------------+-------------------+
only showing top 2 rows



In [42]:
# converting case closed and case late to boolean
case_df = case_df.withColumn("case_late", expr("case_late == 'YES'"))

case_df = case_df.withColumn("case_closed", expr("case_closed == 'YES'"))

In [43]:
# formatting the address column
case_df=case_df.withColumn('request_address', lower(trim('request_address')))

# How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

# How many Stray Animal cases are there?


In [56]:
case_df.filter(case_df.service_request_type == 'Stray Animal').count()

26760

In [45]:
case_df.where(col('service_request_type') == 'Stray Animal').count()

26760

# How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?



In [47]:
case_df.filter(col('service_request_type')

0

In [57]:
(case_df
.filter(col('dept_division') == lit('Field Operations'))
.filter(col('service_request_type') != lit('Officer Standby'))
 
.count()
)

113902

# Convert the council_district column to a string column.



In [50]:
case_df = case_df.withColumn('council_district', col('council_district').cast('string'))

# Extract the year from the case_closed_date column.


In [51]:
case_df.select(year('case_closed_date').alias('Year_case_closed')).show(4)

+----------------+
|Year_case_closed|
+----------------+
|            2018|
|            2018|
|            2018|
|            2018|
+----------------+
only showing top 4 rows



# Convert num_days_late from days to hours in new columns num_hours_late.



In [53]:
case_df.withColumn('hours_late', expr('num_days_late * 24')).show(3,vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 case_due_date        | 2018-01-01 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
 hours_late           | -23964.2102784       
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 2018-01-01 00:46:00  
 case_closed_date     | 2018-01-01 00:46:00  
 case_due_date        | 2018-01-01 00:46:00  
 case_late            | false     

# Join the case data with the source and department data.



In [59]:
case_df = case_df.join(dept_df,'dept_division', 'left')
case_df = case_df.join(source_df, 'source_id','left')

# Are there any cases that do not have a request source?



# What are the top 10 service request types in terms of number of requests?


In [63]:
case_df.groupby('service_request_type').count().sort(desc('count')).show(10)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|           No Pickup|89210|
|Overgrown Yard/Trash|66403|
|        Bandit Signs|32968|
|        Damaged Cart|31163|
|Front Or Side Yar...|28920|
|        Stray Animal|27361|
|Aggressive Animal...|25492|
|Cart Exchange Req...|22608|
|Junk Vehicle On P...|21649|
|     Pot Hole Repair|20827|
+--------------------+-----+
only showing top 10 rows



# What are the top 10 service request types in terms of average days late?


In [64]:
(
case_df.groupBy('service_request_type')
.agg(mean('num_days_late')
.alias('average_late_days'))
.sort(desc('average_late_days'))
.show(10, truncate = False)
)

+--------------------------------------+------------------+
|service_request_type                  |average_late_days |
+--------------------------------------+------------------+
|Zoning: Junk Yards                    |175.9563621042095 |
|Labeling for Used Mattress            |162.43032902285717|
|Record Keeping of Used Mattresses     |153.99724039428568|
|Signage Requied for Sale of Used Mattr|151.63868055333333|
|Storage of Used Mattress              |142.112556415     |
|Zoning: Recycle Yard                  |135.92851612479797|
|Donation Container Enforcement        |131.75610506358706|
|License Requied Used Mattress Sales   |128.79828704142858|
|Traffic Signal Graffiti               |101.79846062200002|
|Complaint                             |72.87050230311695 |
+--------------------------------------+------------------+
only showing top 10 rows



# Does number of days late depend on department?


In [88]:
case_df.groupBy('dept_division')\
.agg(avg('num_days_late').alias('average_late_days'))\
.sort(desc('average_late_days'))\
.show(10, truncate= False)


+-----------------------------+------------------+
|dept_division                |average_late_days |
+-----------------------------+------------------+
|Code Enforcement (Internal)  |135.92851612479797|
|Reservations                 |66.03116319       |
|311 Call Center              |59.737091496300785|
|Director's Office Horizontal |37.570646702950086|
|Engineering Division         |13.43372455586971 |
|Shops                        |9.64126176872269  |
|Tree Crew                    |4.723282812065399 |
|Solid Waste                  |3.5190239198762248|
|Trades                       |3.231977141276932 |
|Clean and Green Natural Areas|1.691468919487805 |
+-----------------------------+------------------+
only showing top 10 rows



In [89]:
case_df.groupBy(case_df.dept_division).agg(avg(case_df.num_days_late).alias('days_late')).sort(desc('days_late')).show()



+--------------------+-------------------+
|       dept_division|          days_late|
+--------------------+-------------------+
|Code Enforcement ...| 135.92851612479797|
|        Reservations|        66.03116319|
|     311 Call Center| 59.737091496300785|
|Director's Office...| 37.570646702950086|
|Engineering Division|  13.43372455586971|
|               Shops|   9.64126176872269|
|           Tree Crew|  4.723282812065399|
|         Solid Waste| 3.5190239198762248|
|              Trades|  3.231977141276932|
|Clean and Green N...|  1.691468919487805|
|              Vector|-1.1206532993223441|
|    Facility License|-1.4126937702216633|
|       Miscellaneous|-1.7218576838926591|
|    Waste Collection| -2.170652238479914|
|     Clean and Green| -2.557154979254144|
|               Brush| -3.985790571457094|
| Food Establishments| -6.971552370451523|
|  Signs and Markings| -7.448628001357712|
|    Shops (Internal)| -8.186267119876439|
|         Storm Water|-14.055678397031881|
+----------

# How do number of days late depend on department and request type

In [91]:
case_df.groupBy('dept_division','service_rerquest_type').agg(avg('num_days_late').alias('days_late')).sort(desc('days_late')).show()



AnalysisException: Column 'service_rerquest_type' does not exist. Did you mean one of the following? [service_request_type, source_username, case_due_date, request_address, case_closed_date, case_opened_date, case_status, standardized_dept_name, standardized_dept_name, case_late, dept_name, dept_name, dept_subject_to_SLA, dept_subject_to_SLA, num_days_late, source_id, case_closed, case_id, council_district, dept_division, SLA_days];
'Aggregate [dept_division#24, 'service_rerquest_type], [dept_division#24, 'service_rerquest_type, avg(num_days_late#22) AS days_late#1481]
+- Project [source_id#28, dept_division#24, case_id#17, case_opened_date#377, case_closed_date#392, case_due_date#407, case_late#469, num_days_late#22, case_closed#484, service_request_type#25, SLA_days#26, case_status#27, request_address#499, council_district#576, dept_name#156, standardized_dept_name#157, dept_subject_to_SLA#158, dept_name#836, standardized_dept_name#837, dept_subject_to_SLA#838, source_username#135]
 

In [84]:
# (
# case_df.groupBy('standardized_dept_name', 'service_request_type')
# .agg(mean('num_days_late')
# .alias('average_late_days'))
# .where(col('average_late_days') > 0)
# .sort(desc('average_late_days'))
# .show(100,truncate = False)
# )